### Imports and utility

In [14]:
# Standard library imports
import os
import sys
import importlib
import numpy as np
import random
import cv2
from tqdm import tqdm

# Add project-specific paths
sys.path.append(os.path.join(os.getcwd(), 'common'))

# Import custom modules
import utility
import embedding
import detection
import grt

# Reload module for updates
importlib.reload(utility)
importlib.reload(embedding)
importlib.reload(detection)
importlib.reload(grt)

<module 'grt' from 'c:\\Users\\Utente\\Documents\\Università\\Master\\MultimediaDataSecurity\\CTM\\multimedia-data-security\\common\\grt.py'>

In [2]:
def robustness(wpsnr):
    if wpsnr >= 53:
        points = 0
    elif wpsnr >= 50:
        points = 1
    elif wpsnr >= 47:
        points = 2
    elif wpsnr >= 44:
        points = 3
    elif wpsnr >= 41:
        points = 4
    elif wpsnr >= 38:
        points = 5
    else:
        points = 6
    return points

def imperceptibility(wpsnr):
    if wpsnr >= 66:
        points = 6
    elif wpsnr >= 62:
        points = 5
    elif wpsnr >= 58:
        points = 4
    elif wpsnr >= 54:
        points = 3
    elif wpsnr >= 50:
        points = 2
    elif wpsnr >= 35:
        points = 1
    else:
        points = 0
    return points

### Embedding

In [3]:
N = 1
images = []
watermarked = []
alpha = 0.48
max_layer = 1
mask = 2
avg = 0
min = 9999999999

# Randomly select N images
img_folder = 'sample_imgs'
img_files =  [f for f in os.listdir(img_folder) if f.endswith(('.bmp'))]
img_files = random.sample(img_files, N)
for file in img_files:
    img_path = os.path.join(img_folder, file)
    images.append(cv2.imread(img_path, 0))
images = np.array(images)

mark = np.load('ammhackati.npy')
   
progress_bar = tqdm(images, desc = "Embedding the watermark in " + str(N) + " images")
for index, img in enumerate(progress_bar):
    watermarked.append(embedding.embedding(img, mark, alpha, max_layer, mask))
    wpsnr = utility.wpsnr(img, watermarked[index])
    avg += wpsnr
    if wpsnr < min:
        min = wpsnr
        
avg = avg / N
print("\nAverage wpsnr: " + str(avg) + ' --> ' + str(imperceptibility(avg)) + ' imperceptibility points')
print("Min wpsnr: " + str(min) + ' --> ' + str(imperceptibility(min)) + ' imperceptibility points')

Embedding the watermark in 1 images: 100%|██████████| 1/1 [00:01<00:00,  1.47s/it]


Average wpsnr: 71.7525091057041 --> 6 imperceptibility points
Min wpsnr: 71.7525091057041 --> 6 imperceptibility points


### Global attacks

1. Blur using gaussian filters

In [ ]:
progress_bar = tqdm(watermarked, desc = "Blurring " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for sigma in np.arange(0, 5, 0.1, dtype = np.float16):
        attacked, attack_name, usd = grt.blur_gauss(wm, sigma)
        detected = detection.detection(images[index], wm, attacked, alpha, max_layer, mask)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0 and wpsnr >= 35:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective... --> 6 robustness points\n")
 
for s in successful_attacks:
    print(s) 

2. Blur using median filters

In [ ]:
progress_bar = tqdm(watermarked, desc = "Blurring " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for height in np.arange(1, 15, 2):
        for width in np.arange(1, 15, 2):
            attacked, attack_name, usd = grt.blur_median(wm, [height, width])
            detected = detection.detection(images[index], wm, attacked, alpha, max_layer, mask)
            wpsnr_attacked = utility.wpsnr(wm, attacked)
            log = {"image" : img_files[index],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}
            history.append(log)
            if detected == 0 and wpsnr >= 35:
                total += wpsnr_attacked
                succ +=1
                successful_attacks.append(log)
                if wpsnr_attacked > max:
                    max = wpsnr_attacked
                break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective... --> 6 robustness points\n")

for s in successful_attacks:
    print(s) 

3. Adding white gaussian noise (AWGN)

In [ ]:
progress_bar = tqdm(watermarked, desc = "Applying AWGN to " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for std in range(0, 110, 10):
        attacked, attack_name, usd = grt.awgn(wm, 0, std, 100)
        detected = detection.detection(images[index], wm, attacked, alpha, max_layer, mask)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0 and wpsnr >= 35:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective... --> 6 robustness points\n")
 
for s in successful_attacks:
    print(s)

4. JPEG compression

In [ ]:
progress_bar = tqdm(watermarked, desc = "Compressing " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for qf in range(0, 100, 2):
        attacked, attack_name, usd = grt.jpeg_compression(wm, (100 - qf))
        detected = detection.detection(images[index], wm, attacked, alpha, max_layer, mask)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0 and wpsnr >= 35:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective... --> 6 robustness points\n")

for s in successful_attacks:
    print(s)

5. Resizing

In [ ]:
progress_bar = tqdm(watermarked, desc = "Resizing " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for scale in np.arange(0, 1, 0.02, dtype = np.float16):
        attacked, attack_name, usd = grt.resize(wm, (1 - scale))
        detected = detection.detection(images[index], wm, attacked, alpha, max_layer, mask)
        wpsnr_attacked = utility.wpsnr(wm, attacked)
        log = {"image" : img_files[index],
                "attack" : attack_name,
                "param" : usd,
                "wpsnr" : wpsnr_attacked,
                "detection" : detected}
        history.append(log)
        if detected == 0 and wpsnr >= 35:
            total += wpsnr_attacked
            succ +=1
            successful_attacks.append(log)
            if wpsnr_attacked > max:
                max = wpsnr_attacked
            break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective... --> 6 robustness points\n")
 
for s in successful_attacks:
    print(s) 

6. Sharpening

In [ ]:
progress_bar = tqdm(watermarked, desc = "Sharpening " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for sigma in np.arange(0, 5, 0.2, dtype = np.float16):
        for a in np.arange(0, 1, 0.1, dtype = np.float16):
            attacked, attack_name, usd = grt.sharp(wm, sigma, a)
            detected = detection.detection(images[index], wm, attacked, alpha, max_layer, mask)
            wpsnr_attacked = utility.wpsnr(wm, attacked)
            log = {"image" : img_files[index],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}
            history.append(log)
            if detected == 0 and wpsnr >= 35:
                total += wpsnr_attacked
                succ +=1
                successful_attacks.append(log)
                if wpsnr_attacked > max:
                    max = wpsnr_attacked
                break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective... --> 6 robustness points\n")
 
for s in successful_attacks:
    print(s) 

### Localized attacks

1. Blur edges using gaussian filters 

In [ ]:
progress_bar = tqdm(watermarked, desc = "Blurring edges of " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for edge_func in range(0, 2):
        for sigma in np.arange(0, 5, 0.1, dtype = np.float16):        
            attacked, attack_name, usd = grt.gauss_edge(wm, sigma, edge_func)
            detected = detection.detection(images[index], wm, attacked, alpha, max_layer, mask)
            wpsnr_attacked = utility.wpsnr(wm, attacked)
            log = {"image" : img_files[index],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}
            history.append(log)
            if detected == 0 and wpsnr >= 35:
                total += wpsnr_attacked
                succ +=1
                successful_attacks.append(log)
                if wpsnr_attacked > max:
                    max = wpsnr_attacked
                break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective... --> 6 robustness points\n")
 
for s in successful_attacks:
    print(s) 

2. Blur edges using median filters

In [ ]:
progress_bar = tqdm(watermarked, desc = "Blurring edges of " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for edge_func in range(0, 2):
        for height in np.arange(1, 15, 2):
            for width in np.arange(1, 15, 2):       
                attacked, attack_name, usd = grt.median_edge(wm, [height, width], edge_func)
                detected = detection.detection(images[index], wm, attacked, alpha, max_layer, mask)
                wpsnr_attacked = utility.wpsnr(wm, attacked)
                log = {"image" : img_files[index],
                        "attack" : attack_name,
                        "param" : usd,
                        "wpsnr" : wpsnr_attacked,
                        "detection" : detected}
                history.append(log)
                if detected == 0 and wpsnr >= 35:
                    total += wpsnr_attacked
                    succ +=1
                    successful_attacks.append(log)
                    if wpsnr_attacked > max:
                        max = wpsnr_attacked
                    break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective... --> 6 robustness points\n")
 
for s in successful_attacks:
    print(s) 

3. Blur flat regions using gaussian filters

In [ ]:
progress_bar = tqdm(watermarked, desc = "Blurring edges of " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for edge_func in range(0, 2):
        for sigma in np.arange(0, 5, 0.1, dtype = np.float16):        
            attacked, attack_name, usd = grt.gauss_flat(wm, sigma, edge_func)
            detected = detection.detection(images[index], wm, attacked, alpha, max_layer, mask)
            wpsnr_attacked = utility.wpsnr(wm, attacked)
            log = {"image" : img_files[index],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}
            history.append(log)
            if detected == 0 and wpsnr >= 35:
                total += wpsnr_attacked
                succ +=1
                successful_attacks.append(log)
                if wpsnr_attacked > max:
                    max = wpsnr_attacked
                break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective... --> 6 robustness points\n")
 
for s in successful_attacks:
    print(s) 

4. Blur flat regions using median filters

In [ ]:
progress_bar = tqdm(watermarked, desc = "Blurring edges of " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for edge_func in range(0, 2):
        for height in np.arange(1, 15, 2):
            for width in np.arange(1, 15, 2):       
                attacked, attack_name, usd = grt.median_flat(wm, [height, width], edge_func)
                detected = detection.detection(images[index], wm, attacked, alpha, max_layer, mask)
                wpsnr_attacked = utility.wpsnr(wm, attacked)
                log = {"image" : img_files[index],
                        "attack" : attack_name,
                        "param" : usd,
                        "wpsnr" : wpsnr_attacked,
                        "detection" : detected}
                history.append(log)
                if detected == 0 and wpsnr >= 35:
                    total += wpsnr_attacked
                    succ +=1
                    successful_attacks.append(log)
                    if wpsnr_attacked > max:
                        max = wpsnr_attacked
                    break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective... --> 6 robustness points\n")
 
for s in successful_attacks:
    print(s) 

5. Adding white gaussian noise (AWGN) to edges

In [ ]:
progress_bar = tqdm(watermarked, desc = "Applying AWGN to edges of " + str(N) + " images")
history = []
successful_attacks = []
total = 0
succ = 0
avg = 0
max = 0

for index, wm in enumerate(progress_bar):
    for edge_func in range(0, 2):
        for std in range(0, 110, 10):      
            attacked, attack_name, usd = grt.awgn_edge(wm, 0, std, 100, edge_func)
            detected = detection.detection(images[index], wm, attacked, alpha, max_layer, mask)
            wpsnr_attacked = utility.wpsnr(wm, attacked)
            log = {"image" : img_files[index],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}
            history.append(log)
            if detected == 0 and wpsnr >= 35:
                total += wpsnr_attacked
                succ +=1
                successful_attacks.append(log)
                if wpsnr_attacked > max:
                    max = wpsnr_attacked
                break

if succ > 0:
    avg = total/succ
    print("\nAverage wpsnr of successful attacks: " + str(avg) + ' --> ' + str(robustness(avg)) + ' robustness points')
    print("Max wpsnr of successful attacks: " + str(max) + ' --> ' + str(robustness(max)) + ' robustness points\n') 
else:
    print("\nThis attack is not very effective... --> 6 robustness points\n")
 
for s in successful_attacks:
    print(s) 

### Combo attacks

1. Resize a JPEG-compressed watermarked image

In [ ]:
attack_name = "resize_jpeg"
usd = {'qf':55, 'scale':1}
attacked = grt.attack(watermarked[0], attack_name, usd)
detected = detection.detection(images[0], watermarked[0], attacked, alpha, max_layer, mask)
wpsnr_attacked = utility.wpsnr(watermarked[0], attacked)
log = {"image" : img_files[0],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}

print(log)

2. Blur a JPEG-compressed watermarked image using gaussian filters

In [ ]:
attack_name = "gauss_jpeg"
usd = {'qf':55, 'sigma':0}
attacked = grt.attack(watermarked[0], attack_name, usd)
detected = detection.detection(images[0], watermarked[0], attacked, alpha, max_layer, mask)
wpsnr_attacked = utility.wpsnr(watermarked[0], attacked)
log = {"image" : img_files[0],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}

print(log)

3. Blur a JPEG-compressed watermarked image using median filters

In [ ]:
attack_name = "median_jpeg"
usd = {'qf':55, 'size':[1,1]}
attacked = grt.attack(watermarked[0], attack_name, usd)
detected = detection.detection(images[0], watermarked[0], attacked, alpha, max_layer, mask)
wpsnr_attacked = utility.wpsnr(watermarked[0], attacked)
log = {"image" : img_files[0],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}

print(log)

4. Blur after AWGN using gaussian filters

In [ ]:
attack_name = "gauss_awgn"
usd = {'mean':0, 'std':20, 'seed':100, 'sigma':0.78}
attacked = grt.attack(watermarked[0], attack_name, usd)
detected = detection.detection(images[0], watermarked[0], attacked, alpha, max_layer, mask)
wpsnr_attacked = utility.wpsnr(watermarked[0], attacked)
log = {"image" : img_files[0],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}

print(log)

5. Blur after AWGN using median filters

In [ ]:
attack_name = "median_awgn"
usd = {'mean':0, 'std':20, 'seed':100, 'size':1}
attacked = grt.attack(watermarked[0], attack_name, usd)
detected = detection.detection(images[0], watermarked[0], attacked, alpha, max_layer, mask)
wpsnr_attacked = utility.wpsnr(watermarked[0], attacked)
log = {"image" : img_files[0],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}

print(log)

### Testing new attacks

In [41]:
importlib.reload(grt)

attack_name = "gauss_dwt"
usd = {'sigma':[0,1,0]}
attacked = grt.attack(watermarked[0], attack_name, usd)
detected = detection.detection(images[0], watermarked[0], attacked, alpha, max_layer, mask)
wpsnr_attacked = utility.wpsnr(watermarked[0], attacked)
log = {"image" : img_files[0],
                    "attack" : attack_name,
                    "param" : usd,
                    "wpsnr" : wpsnr_attacked,
                    "detection" : detected}

print(log)

{'image': '0012.bmp', 'attack': 'gauss_dwt', 'param': {'sigma': [0, 1, 0]}, 'wpsnr': 54.29290487953578, 'detection': 1}
